In [1]:
import sys
sys.path.append("/home/brunoperdigao/repos/Plan-Classification-Topologic-DGL")

In [2]:
# Import Libraries
from topologicpy.DGL import DGL
from topologicpy.Dictionary import Dictionary
from topologicpy.Topology import Topology
from topologicpy.Vertex import Vertex
from topologicpy.Graph import Graph
from topologicpy.Plotly import Plotly
import dgl
import torch
import random
from dgl import save_graphs, load_graphs

In [3]:
path = "/home/brunoperdigao/repos/Plan-Classification-Topologic-DGL/data"
dataset = DGL.DatasetByCSVPath(path, numberOfGraphClasses=0,
                           nodeATTRKey='feat', edgeATTRKey='feat',
                           nodeOneHotEncode=False, nodeFeaturesCategories=[],
                           edgeOneHotEncode=False, edgeFeaturesCategories=[], addSelfLoop=False)
print(dataset)

Done saving data into cached files.
Dataset("GraphDGL", num_graphs=5, save_path=/home/brunoperdigao/.dgl/GraphDGL)


In [4]:
train_data, val_data, test_data = dgl.data.utils.split_dataset(dataset, [0.7, 0.1, 0.2])

In [5]:
train_data.dataset

Dataset("GraphDGL", num_graphs=5, save_path=/home/brunoperdigao/.dgl/GraphDGL)

---

In [6]:
# Contruct a two-layer GNN model
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        self.conv1 = dglnn.SAGEConv(
            in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv(
            in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [7]:
def evaluate(model, graph, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [8]:
# # print(len(dataset.graphs))
# for i, graph in enumerate(dataset.graphs):
#     # print(i)
#     node_features = graph.ndata['feat']
#     # print(node_features)
#     node_labels = graph.ndata['label']
#     # print(node_labels)
#     train_mask = graph.ndata['train_mask']
#     valid_mask = graph.ndata['val_mask']
#     test_mask = graph.ndata['test_mask']
#     n_features = node_features.shape[1]
#     # print(n_features)
#     n_labels = int(node_labels.max().item() + 1)
#     # print(n_features)

In [9]:
n_features = 2
model = SAGE(in_feats=n_features, hid_feats=100, out_feats=11)
opt = torch.optim.Adam(model.parameters())

for epoch in range(50):
    for i, graph in enumerate(dataset.graphs):
        node_features = graph.ndata['feat']
        node_labels = graph.ndata['label']
        train_mask = graph.ndata['train_mask']
        valid_mask = graph.ndata['val_mask']
        test_mask = graph.ndata['test_mask']
        n_features = node_features.shape[1]
        n_labels = int(node_labels.max().item() + 1)
        
        model.train()
        # forward propagation by using all nodes
        logits = model(graph, node_features)
        # compute loss
        loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
        # compute validation accuracy
        acc = evaluate(model, graph, node_features, node_labels, valid_mask)
        # backward propagation
        opt.zero_grad()
        loss.backward()
        opt.step()
        print(loss.item())
    
        # Save model if necessary.  Omitted in this example.

/home/brunoperdigao/venvs/topologicpy/lib/python3.11/site-packages/torch/cuda/__init__.py:619: UserWarning:

Can't initialize NVML



40.43795394897461
61.21820831298828
51.552616119384766
43.12115478515625
29.05818748474121
31.493309020996094
47.15521240234375
39.98308181762695
34.51831817626953
24.111557006835938
25.830028533935547
37.24467468261719
32.29875946044922
29.11652946472168
21.47393798828125
21.67060661315918
29.789813995361328
26.257099151611328
23.333349227905273
18.366331100463867
16.55080223083496
22.83182716369629
19.995750427246094
18.72762680053711
16.37563133239746
13.232219696044922
16.65989112854004
15.030447959899902
15.903231620788574
14.913195610046387
11.176363945007324
12.205742835998535
11.240872383117676
12.899670600891113
11.949955940246582
9.83270263671875
10.796303749084473
9.189092636108398
10.294612884521484
8.3080472946167
7.815880298614502
8.444066047668457
6.737326145172119
7.033985614776611
5.434022426605225
5.689743995666504
7.813319206237793
5.504091739654541
5.044625759124756
3.8160922527313232
4.379909038543701
5.4943389892578125
3.997061014175415
3.5058743953704834
3.138724

In [10]:
DGL.ModelClassifyNodes(model, test_data.dataset)

{'alllabels': [[3, 4, 7, 4, 2, 3, 10, 1, 5, 8, 6, 0, 9, 4],
  [4, 5, 4, 10, 2, 7, 1, 8, 3, 0, 9],
  [0, 8, 9, 1, 4, 10, 2, 10, 7, 4, 5, 3],
  [3, 0, 5, 8, 4, 4, 3, 2, 4, 9, 6, 10, 7, 1],
  [8, 1, 9, 4, 3, 4, 0, 10, 3, 4, 6, 4, 2, 7, 5, 10]],
 'allpredictions': [[4, 4, 7, 4, 2, 3, 10, 1, 5, 8, 6, 0, 9, 4],
  [3, 5, 4, 10, 2, 7, 1, 8, 4, 0, 9],
  [0, 8, 9, 1, 4, 10, 2, 10, 7, 3, 5, 4],
  [3, 0, 5, 8, 4, 4, 3, 2, 4, 9, 6, 10, 3, 1],
  [8, 1, 9, 4, 3, 4, 0, 10, 4, 4, 6, 4, 2, 3, 5, 10]],
 'trainlabels': [[3, 4, 7, 4, 2, 3, 10, 1, 5, 8, 6],
  [4, 5, 4, 10, 2, 7, 1, 8],
  [0, 8, 9, 1, 4, 10, 2, 10, 7],
  [3, 0, 5, 8, 4, 4, 3, 2, 4, 9, 6],
  [8, 1, 9, 4, 3, 4, 0, 10, 3, 4, 6, 4]],
 'trainpredictions': [[4, 4, 7, 4, 2, 3, 10, 1, 5, 8, 6],
  [3, 5, 4, 10, 2, 7, 1, 8],
  [0, 8, 9, 1, 4, 10, 2, 10, 7],
  [3, 0, 5, 8, 4, 4, 3, 2, 4, 9, 6],
  [8, 1, 9, 4, 3, 4, 0, 10, 4, 4, 6, 4]],
 'validationlabels': [[0], [3], [4], [10], [2]],
 'validationpredictions': [[0], [4], [3], [10], [2]],
 'testlabels': 

In [11]:
dictionary = DGL.ModelClassifyNodes(model, test_data.dataset)
# actual = dictionary['alllabels']
actual = []
for l in dictionary['alllabels']:
    actual.extend(l)
    
print(actual)
# predicted = dictionary['allpredictions']
predicted = []
for l in dictionary['allpredictions']:
    predicted.extend(l)
print(predicted)
result = DGL.Accuracy(actual=actual, predicted=predicted)
print(result)
print("Accuracy:", result['accuracy'])

[3, 4, 7, 4, 2, 3, 10, 1, 5, 8, 6, 0, 9, 4, 4, 5, 4, 10, 2, 7, 1, 8, 3, 0, 9, 0, 8, 9, 1, 4, 10, 2, 10, 7, 4, 5, 3, 3, 0, 5, 8, 4, 4, 3, 2, 4, 9, 6, 10, 7, 1, 8, 1, 9, 4, 3, 4, 0, 10, 3, 4, 6, 4, 2, 7, 5, 10]
[4, 4, 7, 4, 2, 3, 10, 1, 5, 8, 6, 0, 9, 4, 3, 5, 4, 10, 2, 7, 1, 8, 4, 0, 9, 0, 8, 9, 1, 4, 10, 2, 10, 7, 3, 5, 4, 3, 0, 5, 8, 4, 4, 3, 2, 4, 9, 6, 10, 3, 1, 8, 1, 9, 4, 3, 4, 0, 10, 4, 4, 6, 4, 2, 3, 5, 10]
{'accuracy': 0.880597, 'correct': 59, 'mask': [False, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, False, True, False, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, False, True, True, True, True, False, True, True], 'size': 67, 'wrong': 8}
Accuracy: 0.880597


In [12]:
cf = DGL.ConfusionMatrix(actual = actual, predicted = predicted)
print(cf)
fig = Plotly.FigureByConfusionMatrix(cf)
Plotly.Show(fig)

[[ 5  0  0  0  0  0  0  0  0  0  0]
 [ 0  5  0  0  0  0  0  0  0  0  0]
 [ 0  0  5  0  0  0  0  0  0  0  0]
 [ 0  0  0  4  2  0  0  2  0  0  0]
 [ 0  0  0  4 12  0  0  0  0  0  0]
 [ 0  0  0  0  0  5  0  0  0  0  0]
 [ 0  0  0  0  0  0  3  0  0  0  0]
 [ 0  0  0  0  0  0  0  3  0  0  0]
 [ 0  0  0  0  0  0  0  0  5  0  0]
 [ 0  0  0  0  0  0  0  0  0  5  0]
 [ 0  0  0  0  0  0  0  0  0  0  7]]
